<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/(240423)_%EC%8B%A4%ED%97%987_%EC%A4%91%EB%B3%B5%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 application_data.csv  'My Drive'      기계학습특론프로젝트   딥러닝
'Colab Notebooks'       기계학습특론   ★논문분석


In [18]:
import pandas as pd
df=pd.read_excel('/content/gdrive/MyDrive/기계학습특론프로젝트/채용공고_보안.xlsx')

# 데이터 정제하기

In [19]:
pip install konlpy

In [20]:
# 직무 구분자 '-'에 짤려나가지 않도록 하기
df['주요업무']=df['주요업무'].str.replace("isms-p",'ISMS_P',regex=True)
df['주요업무']=df['주요업무'].str.replace("ISMP-P",'ISMS_P',regex=True)
df['주요업무']=df['주요업무'].str.replace("PCI-DSS",'PCI_DSS',regex=True)

In [21]:
df.head(3)

,기업명,채용제목,주요업무,자격요건,우대사항
0,마이뮤직테이스트,Backend Engineer,- 성능향상과 Scalability를 위한 최적화- 보안과 데이터 보호- 데이터베이...,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...
1,미미박스(MEMEBOX),DevOps Engineer,ᆞ AWS상 인프라 운영 유지 및 관리ᆞ 배포 및 CI/CD 개발 운영 및 유지ᆞ ...,ᆞ AWS 기반의 Container 환경 구축 및 CI/CD 등 자동화 관련 기솔에...,ᆞ AWS Certificate 보유ᆞ IT 프로젝트 아키텍트 경험ᆞ JAVA/PH...
2,미미박스(MEMEBOX),Database Administrator,"ᆞ Database를 백업, 복원, 모니터링, 튜닝ᆞ 신규 또는 개선 개발 프로젝트...",ᆞ AWS RDS (Mysql) 사용 경험ᆞ Mysql 5년 이상 사용 경험ᆞ Tr...,ᆞ DB 장애 복구 경험ᆞ 1가지 이상 NoSQL 사용 경험


In [22]:
import re
def split_tasks(text):
    split_text = re.split(r'\s*[\-•ㆍᆞ・・\n.]\s*', text) #해당 문자열이 들어간 기준으로 잘라내기
    return [s.strip() for s in split_text if s.strip()]

In [23]:
task_split_result=[]
for task in df['주요업무']:
  result=split_tasks(task)
  task_split_result.append(result)
len(task_split_result) #2661개의 채용공고 정보 수집함

2661

In [24]:
# 직무 정보로 쪼개기
company_list=[]
title_list=[]
qualify_list=[]
prefer_list=[]
task_list=[]

for k in range(df.shape[0]):
  company=df.iloc[k]['기업명']
  title=df.iloc[k]['채용제목']
  qualify=df.iloc[k]['자격요건']
  prefer=df.iloc[k]['우대사항']
  tasks=task_split_result[k]
  for task in tasks:
    company_list.append(company)
    title_list.append(title)
    qualify_list.append(qualify)
    prefer_list.append(prefer)
    task_list.append(task)

rows = zip(company_list, title_list, task_list, qualify_list, prefer_list)
df = pd.DataFrame(rows, columns=['기업명', '채용제목', '주요업무', '지원자격', '우대사항'])
df.head(3)

,기업명,채용제목,주요업무,지원자격,우대사항
0,마이뮤직테이스트,Backend Engineer,성능향상과 Scalability를 위한 최적화,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...
1,마이뮤직테이스트,Backend Engineer,보안과 데이터 보호,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...
2,마이뮤직테이스트,Backend Engineer,"데이터베이스 솔루션 디자인및 구현음악과 기술에 대한 열정이 넘치는 사람들로 가득한,...",- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...


In [25]:
df.shape #총 14849개의 보안직무 정보 찾아냄

(14849, 5)

# 데이터 전처리

In [26]:
df['업무길이'] = df['주요업무'].apply(len)
df=df[(df['업무길이']>=5) & (df['업무길이']<=50)] #너무짧거나 긴 문장 삭제

In [27]:
df.shape

(12556, 6)

In [28]:
pip install konlpy

In [29]:
# 중복제거 (재공고 사항 삭제)
df=df.drop_duplicates()

## 조사, 어미, 구두점 제거, 어간추출 - Okt 중심으로

In [30]:
from konlpy.tag import Okt
okt=Okt()

#조사, 어미, 구두점 제거, 어간 추출
def text_clean(text):
  clean_text=[]
  for word in okt.pos(text, stem=True):
    if word[1] not in ['Josa', 'Eomi', 'Punction']:
      clean_text.append(word[0])
  return " ".join(clean_text)

clean_text=[]
for text in df['주요업무']:
  result=text_clean(text)
  result=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ,a-z, A-Z]", " ", result)
  clean_text.append(result)

clean_text=pd.DataFrame(clean_text, columns=['주요업무_C'])
df=df.reset_index(drop=True)
df= pd.concat([df, clean_text], axis=1)

In [31]:
df.head(3)

,기업명,채용제목,주요업무,지원자격,우대사항,업무길이,주요업무_C
0,마이뮤직테이스트,Backend Engineer,성능향상과 Scalability를 위한 최적화,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,25,성능 향상 Scalability 를 위 최적화
1,마이뮤직테이스트,Backend Engineer,보안과 데이터 보호,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,10,보안 데이터 보호
2,마이뮤직테이스트,Backend Engineer,우리는 콘서트 시장의 패러다임을 획기적으로 바꿀 수 있는 서비스를 만들고 있습니다,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,45,우리 콘서트 시장 패러다임 획기 적 바꾸다 수 있다 서비스 만들다 있다


In [32]:
df['주요업무_C']=df['주요업무_C'].astype('str')

In [33]:
# 불용어 제거
def remove_stopwords(text):
  tokens=text.split(' ')
  stops=set(['은', '는', '이', '가', '아', '하', '들', '것', '의', '있', '되', '수', '보', '주', '등', '한', '에','합니다', '하는', '할', '하는', '한다', '그리고', '입니다', '그', '등', '이런', '것', '제', '더', '및'])
  meaningful_words=[w for w in tokens if not w in stops]
  return ' '.join(meaningful_words)
df['주요업무_C']=df['주요업무_C'].map(remove_stopwords)

In [34]:
df.head(10)

,기업명,채용제목,주요업무,지원자격,우대사항,업무길이,주요업무_C
0,마이뮤직테이스트,Backend Engineer,성능향상과 Scalability를 위한 최적화,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,25,성능 향상 Scalability 를 위 최적화
1,마이뮤직테이스트,Backend Engineer,보안과 데이터 보호,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,10,보안 데이터 보호
2,마이뮤직테이스트,Backend Engineer,우리는 콘서트 시장의 패러다임을 획기적으로 바꿀 수 있는 서비스를 만들고 있습니다,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,45,우리 콘서트 시장 패러다임 획기 적 바꾸다 있다 서비스 만들다 있다
3,마이뮤직테이스트,Backend Engineer,"이에 따라 열정적이고 창의적으로, 즐겁게 함께 일할 사람들을 찾고 있습니다",- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,41,"따르다 열정 적 창의 적 , 즐겁다 함께 일 하다 사람 찾다 있다"
4,마이뮤직테이스트,Backend Engineer,우리는 완전하지 않습니다,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,13,우리 완전하다 않다
5,마이뮤직테이스트,Backend Engineer,많은 문제가 있음을 인정합니다,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,16,많다 문제 있다 음 인정 하다
6,마이뮤직테이스트,Backend Engineer,우리는 개선하기 위해 끊임없이 노력 중이며 아직 해결해야할 문제들이 많이 남아있습니다,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,47,우리 개선 하다 위해 끊임없다 노력 중이 아직 해결 하다 하다 문제 많이 남다
7,마이뮤직테이스트,Backend Engineer,"그래서 아주 특별한 개발팀에서 일할 똑똑하고, 자발적으로 동기가 부여된 개발자를 찾습니다",- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,49,"그래서 아주 특별하다 개발 팀 일 하다 똑똑하다 , 자발 적 동기 부여 되다 개발자 찾다"
8,미미박스(MEMEBOX),DevOps Engineer,AWS상 인프라 운영 유지 및 관리,ᆞ AWS 기반의 Container 환경 구축 및 CI/CD 등 자동화 관련 기솔에...,ᆞ AWS Certificate 보유ᆞ IT 프로젝트 아키텍트 경험ᆞ JAVA/PH...,19,AWS 상 인프라 운영 유지 관리
9,미미박스(MEMEBOX),DevOps Engineer,배포 및 CI/CD 개발 운영 및 유지,ᆞ AWS 기반의 Container 환경 구축 및 CI/CD 등 자동화 관련 기솔에...,ᆞ AWS Certificate 보유ᆞ IT 프로젝트 아키텍트 경험ᆞ JAVA/PH...,21,배포 CI CD 개발 운영 유지


In [35]:
#채용제목과 합치기
df['text']=df['채용제목']+" "+df['주요업무_C']

In [36]:
df['text']=df['text'].astype('str')

In [ ]:
df2=df[df['text'].str.contains('보안|보호|security')]

In [ ]:
df2.shape

(7189, 8)

In [ ]:
df2.head(10)

,기업명,채용제목,주요업무,지원자격,우대사항,업무길이,주요업무_C,text
1,마이뮤직테이스트,Backend Engineer,보안과 데이터 보호,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,10,보안 데이터 보호,Backend Engineer 보안 데이터 보호
10,미미박스(MEMEBOX),DevOps Engineer,서비스망 보안 확보,ᆞ AWS 기반의 Container 환경 구축 및 CI/CD 등 자동화 관련 기솔에...,ᆞ AWS Certificate 보유ᆞ IT 프로젝트 아키텍트 경험ᆞ JAVA/PH...,10,서비스 망 보안 확보,DevOps Engineer 서비스 망 보안 확보
16,미미박스(MEMEBOX),Database Administrator,회원 DB에 대한 보안 관련 업무를 지원,ᆞ AWS RDS (Mysql) 사용 경험ᆞ Mysql 5년 이상 사용 경험ᆞ Tr...,ᆞ DB 장애 복구 경험ᆞ 1가지 이상 NoSQL 사용 경험,22,회원 DB 대한 보안 관련 업무 지원,Database Administrator 회원 DB 대한 보안 관련 업무 지원
20,보닥(아이지넷),백엔드 개발자,서버 정책 수립 가능자 (보안이나 백업 등),•\tAWS 에 능숙한 사람•\t서버 자동화 스크립트 및 전사 통합 개발환경 구성에...,• 자유롭고 창의적으로 업무할 수 있습니다.,24,서버 정책 수립 가능 자 ( 보안 백업 ),백엔드 개발자 서버 정책 수립 가능 자 ( 보안 백업 )
23,이스트소프트(ESTsoft),보안서비스 서버 개발자,신규 프로젝트 서버 개발 (40%),• C++ 에 대한 이해가 있는 분 • 네트워크 프로그래밍에 대한 기본 지식이 있...,"• 멀티 플랫폼 개발(Windows, Linux 등)이 가능한 분 우대 • 대용량...",19,신규 프로젝트 서버 개발 ( 40% ),보안서비스 서버 개발자 신규 프로젝트 서버 개발 ( 40% )
24,이스트소프트(ESTsoft),보안서비스 서버 개발자,신규 프로젝트 관련 DB 설계 및 튜닝 작업 (30%),• C++ 에 대한 이해가 있는 분 • 네트워크 프로그래밍에 대한 기본 지식이 있...,"• 멀티 플랫폼 개발(Windows, Linux 등)이 가능한 분 우대 • 대용량...",30,신규 프로젝트 관련 DB 설계 튜닝 작업 ( 30% ),보안서비스 서버 개발자 신규 프로젝트 관련 DB 설계 튜닝 작업 ( 30% )
25,뱅크샐러드,보안 엔지니어,뱅크샐러드 서비스의 안정적 운영을 위한 보안 관리,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,27,뱅크 샐러드 서비스 안정 적 운영 위 보안 관리,보안 엔지니어 뱅크 샐러드 서비스 안정 적 운영 위 보안 관리
26,뱅크샐러드,보안 엔지니어,뱅크샐러드 시스템 및 서비스 취약점 점검 및 분석,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,27,뱅크 샐러드 시스템 서비스 취약점 점검 분석,보안 엔지니어 뱅크 샐러드 시스템 서비스 취약점 점검 분석
27,뱅크샐러드,보안 엔지니어,뱅크샐러드 이용자 개인정보 보호 관련 업무,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,23,뱅크 샐러드 이용자 개인정보 보호 관련 업무,보안 엔지니어 뱅크 샐러드 이용자 개인정보 보호 관련 업무
28,뱅크샐러드,보안 엔지니어,뱅크샐러드 정보보호 관리체계 구축 및 관리,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,23,뱅크 샐러드 정보보호 관리 체계 구축 관리,보안 엔지니어 뱅크 샐러드 정보보호 관리 체계 구축 관리


# 단어 벡터화

## CountVectorizer+K-means

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
corpus=df2['text']
bow=vector.fit_transform(corpus)

In [ ]:
columns=[]
for k, v in sorted(vector.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)
import pandas as pd
Bow=pd.DataFrame(bow.toarray(), columns=columns)

In [ ]:
from sklearn.cluster import KMeans
n_clusters=10
kmeans=KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(Bow)
prediction=kmeans.predict(Bow)
df2['cluster']=prediction

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-32-5a43fcecdf22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster']=prediction


In [ ]:
kmeans.inertia_

63097.81685086642

In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(Bow, prediction)
score

0.03841564718261289

#TF-IDF+Kmeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus=df2['text']
tfidf=TfidfVectorizer().fit(corpus)
columns=[]
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)
import pandas as pd
tfidf=pd.DataFrame(tfidf.transform(corpus).toarray(), columns=columns)

In [ ]:
from sklearn.cluster import KMeans
n_clusters=10
kmeans=KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(tfidf)
prediction=kmeans.predict(tfidf)
df2['cluster']=prediction

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-36-278be200108b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster']=prediction


In [ ]:
kmeans.inertia_

6478.556561268265

In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(tfidf, prediction)
score

0.018534573049023048

# TF-IDF + 한국어토큰화+KMeans

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Kkma
import numpy as np
from sklearn.cluster import KMeans

kkma=Kkma()

text_list=df2['text']

def tokenize_korean_text(text_list):
    return [" ".join(kkma.morphs(text)) for text in text_list]

tokenized_data = tokenize_korean_text(text_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer().fit(tokenized_data)
columns=[]
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)
import pandas as pd
tfidf=pd.DataFrame(tfidf.transform(tokenized_data).toarray(), columns=columns)

In [ ]:
from sklearn.cluster import KMeans
n_clusters=10
kmeans=KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(tfidf)
prediction=kmeans.predict(tfidf)
df2['cluster']=prediction

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-42-278be200108b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster']=prediction


In [ ]:
kmeans.inertia_

6170.384979221362

In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(tfidf, prediction)
score

0.01776123450820275

# TF-idf(sublinear_tf 적용) +한국어토큰화+K-means

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Kkma
import numpy as np
from sklearn.cluster import KMeans

kkma=Kkma()

text_list=df2['text']

def tokenize_korean_text(text_list):
    return [" ".join(kkma.morphs(text)) for text in text_list]

tokenized_data = tokenize_korean_text(text_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(sublinear_tf=True).fit(tokenized_data) #sublinear_tf : smoothing 효과
columns=[]
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)
import pandas as pd
tfidf=pd.DataFrame(tfidf.transform(tokenized_data).toarray(), columns=columns)

In [ ]:
from sklearn.cluster import KMeans
n_clusters=10
kmeans=KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(tfidf)
prediction=kmeans.predict(tfidf)
df2['cluster']=prediction

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-48-278be200108b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster']=prediction


In [ ]:
kmeans.inertia_

6235.233320089883

In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(tfidf, prediction)
score

0.022068747198561185

# TF-IDF 조정해보기

In [ ]:
from konlpy.tag import Kkma
import numpy as np
from sklearn.cluster import KMeans

kkma=Kkma()

text_list=df2['text']

def tokenize_korean_text(text_list):
    return [" ".join(kkma.morphs(text)) for text in text_list]

tokenized_data = tokenize_korean_text(text_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=1000, min_df=5, max_df=0.9).fit(tokenized_data) #설정값 변경해보기
columns=[]
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)
import pandas as pd
tfidf=pd.DataFrame(tfidf.transform(tokenized_data).toarray(), columns=columns)

In [ ]:
from sklearn.cluster import KMeans
n_clusters=10
kmeans=KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(tfidf)
prediction=kmeans.predict(tfidf)
df2['cluster']=prediction

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-65-278be200108b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster']=prediction


In [ ]:
kmeans.inertia_

6080.693591904008

In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(tfidf, prediction)

In [ ]:
score

0.030802381593239503

In [ ]:
df2.head(10)

,기업명,채용제목,주요업무,지원자격,우대사항,업무길이,주요업무_C,text,cluster
1,마이뮤직테이스트,Backend Engineer,보안과 데이터 보호,- 3년이상의 Python 외의 언어로 웹서버 개발 혹은 Python으로 웹서버 개...,- 새로운 기술에 대한 적극적인 관심- 기존 솔루션을 유지/보수하면서 적극적으로 문...,10,보안 데이터 보호,Backend Engineer 보안 데이터 보호,2
10,미미박스(MEMEBOX),DevOps Engineer,서비스망 보안 확보,ᆞ AWS 기반의 Container 환경 구축 및 CI/CD 등 자동화 관련 기솔에...,ᆞ AWS Certificate 보유ᆞ IT 프로젝트 아키텍트 경험ᆞ JAVA/PH...,10,서비스 망 보안 확보,DevOps Engineer 서비스 망 보안 확보,2
16,미미박스(MEMEBOX),Database Administrator,회원 DB에 대한 보안 관련 업무를 지원,ᆞ AWS RDS (Mysql) 사용 경험ᆞ Mysql 5년 이상 사용 경험ᆞ Tr...,ᆞ DB 장애 복구 경험ᆞ 1가지 이상 NoSQL 사용 경험,22,회원 DB 대한 보안 관련 업무 지원,Database Administrator 회원 DB 대한 보안 관련 업무 지원,2
20,보닥(아이지넷),백엔드 개발자,서버 정책 수립 가능자 (보안이나 백업 등),•\tAWS 에 능숙한 사람•\t서버 자동화 스크립트 및 전사 통합 개발환경 구성에...,• 자유롭고 창의적으로 업무할 수 있습니다.,24,서버 정책 수립 가능 자 ( 보안 백업 ),백엔드 개발자 서버 정책 수립 가능 자 ( 보안 백업 ),8
23,이스트소프트(ESTsoft),보안서비스 서버 개발자,신규 프로젝트 서버 개발 (40%),• C++ 에 대한 이해가 있는 분 • 네트워크 프로그래밍에 대한 기본 지식이 있...,"• 멀티 플랫폼 개발(Windows, Linux 등)이 가능한 분 우대 • 대용량...",19,신규 프로젝트 서버 개발 ( 40% ),보안서비스 서버 개발자 신규 프로젝트 서버 개발 ( 40% ),8
24,이스트소프트(ESTsoft),보안서비스 서버 개발자,신규 프로젝트 관련 DB 설계 및 튜닝 작업 (30%),• C++ 에 대한 이해가 있는 분 • 네트워크 프로그래밍에 대한 기본 지식이 있...,"• 멀티 플랫폼 개발(Windows, Linux 등)이 가능한 분 우대 • 대용량...",30,신규 프로젝트 관련 DB 설계 튜닝 작업 ( 30% ),보안서비스 서버 개발자 신규 프로젝트 관련 DB 설계 튜닝 작업 ( 30% ),2
25,뱅크샐러드,보안 엔지니어,뱅크샐러드 서비스의 안정적 운영을 위한 보안 관리,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,27,뱅크 샐러드 서비스 안정 적 운영 위 보안 관리,보안 엔지니어 뱅크 샐러드 서비스 안정 적 운영 위 보안 관리,2
26,뱅크샐러드,보안 엔지니어,뱅크샐러드 시스템 및 서비스 취약점 점검 및 분석,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,27,뱅크 샐러드 시스템 서비스 취약점 점검 분석,보안 엔지니어 뱅크 샐러드 시스템 서비스 취약점 점검 분석,4
27,뱅크샐러드,보안 엔지니어,뱅크샐러드 이용자 개인정보 보호 관련 업무,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,23,뱅크 샐러드 이용자 개인정보 보호 관련 업무,보안 엔지니어 뱅크 샐러드 이용자 개인정보 보호 관련 업무,2
28,뱅크샐러드,보안 엔지니어,뱅크샐러드 정보보호 관리체계 구축 및 관리,• 기업 보안 관리 2년 이상의 경력자 • 기업 보안 시스템/체계 구축 및 운영 경...,• ISMS 등 보안 관련 인증 진행 경험자 • 보안 관련 수상 경력자 • 보안 관...,23,뱅크 샐러드 정보보호 관리 체계 구축 관리,보안 엔지니어 뱅크 샐러드 정보보호 관리 체계 구축 관리,2
